In [4]:
from pytorch_retinanet.anchors import AnchorGenerator
from pytorch_retinanet.models import Retinanet
from pytorch_retinanet.layers import get_backbone
import torch

In [7]:
model = Retinanet(num_classes=2, backbone_kind='resnet18')
print(model)

Retinanet(
  (backbone): BackBone(
    (backbone): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): Batc

In [8]:
z = torch.rand([3, 3, 600, 600])
z.shape

torch.Size([3, 3, 600, 600])

In [9]:
model.eval()
outputs = model(z)

Activations shape :  torch.Size([69354, 4])
Anchors Shape:  torch.Size([68229, 4])


RuntimeError: The size of tensor a (69354) must match the size of tensor b (68229) at non-singleton dimension 0